In [117]:
import numpy as np, pandas as pd

In [118]:
!wget https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv

--2019-11-22 09:09:53--  https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.206.128, 2a00:1450:400c:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.206.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/csv]
Saving to: ‘bbc-text.csv.1’

bbc-text.csv.1      100%[===================>]   4.82M  4.24MB/s    in 1.1s    

2019-11-22 09:09:56 (4.24 MB/s) - ‘bbc-text.csv.1’ saved [5057493/5057493]



In [437]:
Text=pd.read_csv("bbc-text.csv")
Text.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [438]:
Text.text[0]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

In [120]:
from sklearn.preprocessing import LabelEncoder as le

In [121]:
Text['category']=le().fit_transform(Text['category'])

In [122]:
import string

In [123]:
from nltk.corpus import stopwords

In [124]:
def text_cleaner(message):
    nopunc=[char for char in message if char not in string.punctuation]
    nopunc=''.join(nopunc).split()
    nopunc=[word for word in nopunc if word.lower() not in stopwords.words('english')]
    return nopunc

In [125]:
from sklearn.feature_extraction.text import CountVectorizer

In [299]:
Text_tranformer=CountVectorizer(analyzer=text_cleaner, max_features=100, binary=True).fit(Text['text'])

In [300]:
bow1=Text_tranformer.transform(Text['text'])

In [301]:
bow1.shape

(2225, 100)

In [302]:
print(bow1.toarray())

[[1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [1 1 0 ... 0 0 0]
 [1 0 0 ... 1 0 0]
 [0 0 0 ... 1 1 0]]


In [303]:
Data=pd.DataFrame(data=bow1.toarray())
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1,0,0,0,1,1,1,1,0,0,...,1,1,0,1,0,1,0,0,0,1
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [304]:
Data['target']=Text['category']

In [305]:
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,target
0,1,0,0,0,1,1,1,1,0,0,...,1,0,1,0,1,0,0,0,1,4
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,3
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,3
4,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [287]:
#import ipdb

In [377]:
class Naive_Bayes(object):
    def __init__(self, features, target):
        self.features=features
        self.target=target
        
    def Phi_ki(self, i, k):
        Res1=np.sum((self.features.iloc[:, i] == 1) & (self.target==k))+1
        Res2=self.target.nunique()
        Res3=np.sum(self.target==k)
        return np.float64(Res1/(Res2+Res3))
    
    def P_Xi_given_y_equal_k(self, i, k, X_i):
        if X_i==1:
            return self.Phi_ki(i, k)
        else:
            return 1-self.Phi_ki(i,k)
        
    def P_y(self, j):
        R=np.sum(self.target==j)/len(self.target)
        return R
    
    def P_Xi(self, i, X_i):
        if X_i==1:
            return np.sum([np.float64(self.Phi_ki(i, k)*self.P_y(k)) \
                                   for k in self.target.unique()])
        else:
            return np.sum([(1-np.float64(self.Phi_ki(i, k))*self.P_y(k)) \
                           for k in self.target.unique()])
    
    def P_y_equal_k_given_Xi(self, k, X):
        Res1=np.prod([np.float64(self.P_Xi_given_y_equal_k(i, k, X[i]))\
            for i in range(len(X))])
        Res2=self.P_y(k)
        Res3=1.0/np.prod([np.float64(self.P_Xi(i, X[i])) for i in range(len(X))])
        return Res1*Res2*Res3
    
    def train_and_predict(self, X):
        predictions=[]
        for j in range(X.shape[0]):
            if j % 20 == 0:
                print("Percentage to completion: {}%".format(100*j/X.shape[0]))
            List=[]
            for k in range(len(self.target.unique())):
                List.append(self.P_y_equal_k_given_Xi(k, X.iloc[j, :]))
            predictions.append(np.argmax(List))
            #print("Size of the predictions: ", len(prediction))
        return np.array(predictions)

In [378]:
from sklearn.model_selection import train_test_split

In [379]:
Train_features, Test_features, Train_target, Test_target = train_test_split(Data.iloc[:, :-1],\
        Data.iloc[:, -1], test_size=0.4, random_state=101)

In [380]:
NB=Naive_Bayes(Train_features, Train_target)

In [382]:
Pred=NB.train_and_predict(Test_features)

In [428]:
accuracy=(np.array(Test_target)==Pred).sum()/len(Test_target)
print("Accuracy: {}%".format(100*accuracy))

Accuracy: 78.08988764044943%


## Comparison with the built-in BernoulliNB

In [325]:
from sklearn.naive_bayes import BernoulliNB as BNB

In [ ]:
BernNB=BNB()
BernNB.fit(Train_features, Train_target)

In [332]:
BernPred=BernNB.predict(Test_features)

In [333]:
Accuracy=(np.array(Test_target)==BernPred).sum()/len(Test_target)
Accuracy*100

77.97752808988764

## Gaussian Discriminant Analysis

In [432]:
class GDAMulticlass(object):
    
    def __init__(self, features, target):
        self.features=features
        self.target=target
        
    def train(self):
        combined=pd.concat([self.features, self.target], axis=1)
        self.mu_s=[combined[combined['target']==j].drop('target', axis=1).mean(axis=0)\
                   for j in range(len(self.target.unique()))]
        self.phi_s=(1.0/len(self.target))*np.array([self.target[self.target==j].count() \
                                    for j in range(len(self.target.unique()))])
        sigma=np.matrix(np.zeros([self.features.shape[1], self.features.shape[1]]))
        for i in range(self.target.shape[0]):
            sigma += np.dot(np.matrix(self.features.iloc[i, :]-\
            self.mu_s[self.target.iloc[i]]).T, \
            np.matrix(self.features.iloc[i, :]-self.mu_s[self.target.iloc[i]]))
        
        self.sigma=(1.0/self.target.shape[0])*sigma
        
    def P_y(self, y, phi_s):
        return phi_s[y]
    
    def P_x_given_y(self, sigma, x, mu):
        comp1 = 1.0/(np.sqrt((2*np.pi)**self.features.shape[1]) * np.sqrt(\
                                                    np.linalg.det(sigma)))
        comp2 = float(np.exp(np.dot(-0.5*np.dot(x-mu, np.linalg.inv(sigma)), x-mu)))
        return comp1*comp2
        
    def predict(self, X):
        predictions=[]
        for i in range(X.shape[0]):
            Prob=[self.P_x_given_y(self.sigma, X.iloc[i, :], self.mu_s[j])*self.P_y(j,\
                    self.phi_s) for j in range(len(self.target.unique()))]
            
            predictions.append(np.argmax(Prob))
            
        return np.array(predictions)
            

In [433]:
MulticlassGDA=GDAMulticlass(Train_features, Train_target)

In [434]:
MulticlassGDA.train()

In [435]:
GDA_Pred=MulticlassGDA.predict(Test_features)

In [436]:
AccuracyGDA=(np.array(Test_target)==GDA_Pred).sum()/len(Test_target)
AccuracyGDA*100

79.21348314606742